<a href="https://colab.research.google.com/github/zaia-majestoso/trab/blob/main/garcons_cozinheiros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import threading
import time


class Restaurante:
    def __init__(self, num_garcons, num_cozinheiros):
        self.pedidos = []
        self.total_pedidos = 0
        self.max_pedidos = 4
        self.condicao = threading.Condition()
        self.garcons = [Garcom(i, self) for i in range(num_garcons)]
        self.cozinheiros = [Cozinheiro(i, self) for i in range(num_cozinheiros)]

    def abrir(self):
        for garcom in self.garcons:
            garcom.start()
        for cozinheiro in self.cozinheiros:
            cozinheiro.start()

    def fechar(self):
        for garcom in self.garcons:
            garcom.join()
        with self.condicao:
            self.condicao.notify_all()
        for cozinheiro in self.cozinheiros:
            cozinheiro.join(timeout=4)
            if cozinheiro.is_alive():
                break

class Garcom(threading.Thread):
    def __init__(self, id, restaurante):
        super().__init__()
        self.id = id
        self.restaurante = restaurante

    def run(self):
        for i in range(10):
            with self.restaurante.condicao:
                while len(self.restaurante.pedidos) == self.restaurante.max_pedidos:
                    self.restaurante.condicao.wait()
                pedido = (self.id, i)
                self.restaurante.pedidos.append(pedido)
                self.restaurante.total_pedidos += 1
                print(f'Garçom {self.id} fez o pedido {i}')
                self.restaurante.condicao.notify_all()
            time.sleep(3)

class Cozinheiro(threading.Thread):
    def __init__(self, id, restaurante):
        super().__init__()
        self.id = id
        self.restaurante = restaurante

    def run(self):
        while True:
            with self.restaurante.condicao:
                while not self.restaurante.pedidos:
                    self.restaurante.condicao.wait()
                if self.restaurante.total_pedidos == 0:
                    break
                pedido = self.restaurante.pedidos.pop(0)
                self.restaurante.total_pedidos -= 1
                print(f'Cozinheiro {self.id} preparou o pedido {pedido[1]} do garçom {pedido[0]}')
                self.restaurante.condicao.notify_all()
            if self.restaurante.total_pedidos == 0:
                break
            time.sleep(3)

if __name__ == "__main__":
    restaurante = Restaurante(num_garcons=5, num_cozinheiros=1)
    inicio = time.time()
    restaurante.abrir()
    restaurante.fechar()
    fim = time.time()
    print(f'Tempo total com 1 cozinheiro: {fim - inicio:.2f} segundos')
    print("\n"*3)
    print("ADICIONANDO MAIS UM COZINHEIRO")
    print("\n")
    restaurante = Restaurante(num_garcons=5, num_cozinheiros=2)
    inicio = time.time()
    restaurante.abrir()
    restaurante.fechar()
    fim = time.time()
    print(f'Tempo total com 2 cozinheiros: {fim - inicio:.2f} segundos')


Garçom 0 fez o pedido 0
Garçom 3 fez o pedido 0
Garçom 2 fez o pedido 0
Garçom 1 fez o pedido 0
Cozinheiro 0 preparou o pedido 0 do garçom 0
Garçom 4 fez o pedido 0
Cozinheiro 0 preparou o pedido 0 do garçom 3
Garçom 4 fez o pedido 1
Cozinheiro 0 preparou o pedido 0 do garçom 2
Garçom 1 fez o pedido 1
Cozinheiro 0 preparou o pedido 0 do garçom 1
Garçom 2 fez o pedido 1
Cozinheiro 0 preparou o pedido 0 do garçom 4
Garçom 0 fez o pedido 1
Cozinheiro 0 preparou o pedido 1 do garçom 4
Garçom 4 fez o pedido 2
Cozinheiro 0 preparou o pedido 1 do garçom 1
Garçom 1 fez o pedido 2
Cozinheiro 0 preparou o pedido 1 do garçom 2
Garçom 2 fez o pedido 2
Cozinheiro 0 preparou o pedido 1 do garçom 0
Garçom 4 fez o pedido 3
Cozinheiro 0 preparou o pedido 2 do garçom 4
Garçom 1 fez o pedido 3
Cozinheiro 0 preparou o pedido 2 do garçom 1
Garçom 0 fez o pedido 2
Cozinheiro 0 preparou o pedido 2 do garçom 2
Garçom 0 fez o pedido 3
Cozinheiro 0 preparou o pedido 3 do garçom 4
Garçom 4 fez o pedido 4
Cozinhe